In [ ]:
from pathlib import Path

import numpy as np

from astropy.nddata import CCDData
from astropy.coordinates import SkyCoord
from astrowidgets import ImageWidget

from glowing_waffles.photometry import detect_sources, faster_sigma_clip_stats, find_too_close, clipped_sky_per_pix_stats, add_to_photometry_table, photometry_on_directory

## Settings that need to be filled in by user 

###  Fill in the names below 

For the directory name you can use either a relative or absolute path.

In [ ]:
directory_with_images = 'path/to/my/images'

#This could be any image from roughly the middle of the sequence.
sample_image_for_finding_stars = 'sample_image_name.fit'

# This only used to center the image viewer on the object. If the object you have can't be looked
# up in simbad this cell will raise an error. If it raises an error leave the object of interest as an empty string.
object_of_interest = 'my_object'

if object_of_interest:
    obj_coords = SkyCoord.from_name(object_of_interest)
    
# The name of your photometry file should include the night number (e.g. 2018-05-21), the object name 
# (e.g. kelt-16b) and the aperture radius (e.g. 12), so maybe: 2018-05-21-kelt-16b-aperture-12
name_of_output_file = 'date-my-object-aperture' + '.fits'

## Set FWHM, pick aperture and annulus sizes

Using the `viewer-seeing-profile` notebook, determine what you want the FWHM should be set to. You want an aperture about 2×FWHM, an annulus with an inner radius at least 10 pixels larger than that, and outer radius about 15 pixels larger than the inner.

In [ ]:
# FWHM is used in the source detection algorithm
FWHM = 7

aperture_rad = 12
inner_annulus = aperture_rad + 15
outer_annulus = inner_annulus + 15

## Choose detection threshold for star detection

This is the number of standard deviations  above the the noise an object must be to be detected.

(The standard deviation maybe estimated with the median absolute deviation).

In [ ]:
sigma_detect = 10

### Set some camera properties

In [ ]:
max_adu = 45000
gain = 1.47
read_noise = 9.42
dark_current = 0.014

# The remainder of notebook can simply be run

You can either "Run All Below" from the Cell menu or execute each cell individually.

In [ ]:
def source_detection(directory, sample_image, FWHM, sigma, aperture):
    #Set pixels that are above point where CCD becomes non-linear to invalid value
    ccd = CCDData.read(str(Path(directory) / sample_image))
    ccd.data[ccd.data > max_adu] = np.nan
    stars = detect_sources(ccd, FWHM, sigma)
    
    #list of stars in RA/Dec so we can find them in each image
    star_locs = ccd.wcs.all_pix2world(stars['xcentroid'], stars['ycentroid'], 0)
    
    #check for stars closer than 2 × aperture radius and remove from list
    too_close = find_too_close(star_locs, aperture)
    star_locs = (star_locs[0][~too_close], star_locs[1][~too_close])
    
    #make up a unique ID for each star
    star_ids = np.arange(len(star_locs[0])) + 1
    
    return ccd, stars, star_locs, too_close, star_ids

In [ ]:
ccd, stars, star_locs, too_close, star_ids = source_detection(directory_with_images, sample_image_for_finding_stars, FWHM, sigma_detect, aperture_rad)

## We are ready to do photometry on all of the images

In [ ]:
all_phot = photometry_on_directory(directory_with_images, object_of_interest,
                                   star_locs, aperture_rad,
                                   inner_annulus, outer_annulus,
                                   max_adu, star_ids,
                                   gain, read_noise, dark_current)

### Display the image

You MUST run the cells below the image display in order to have a picture show up.

In [ ]:
iw = ImageWidget()
iw

In [ ]:
iw.load_fits(str(Path(directory_with_images) / sample_image_for_finding_stars))

In [ ]:
if object_of_interest:
    iw.center_on(SkyCoord.from_name(object_of_interest))

In [ ]:
iw.reset_markers()
iw.marker = {'type': 'circle', 'color': 'lightgreen', 'radius': 10}
iw.add_markers(stars, x_colname='xcentroid', y_colname='ycentroid')

In [ ]:
iw.marker = {'type': 'circle', 'color': 'red', 'radius': 20}
iw.add_markers(stars[too_close], x_colname='xcentroid', y_colname='ycentroid')

### Write out the file!

In [ ]:
all_phot.write(name_of_output_file)